In [1]:
import os
import pandas as pd
import optuna
from qiskit.circuit import QuantumCircuit

import qtpu

In [2]:
NUM_QUBITS = 200
SUBCIRCUIT_SIZE = 60
CSV_FILE = "cost.csv"

In [3]:
from qtpu.compiler.compiler import trial_to_hybrid_tn

def _min_cost_htn(study: optuna.Study) -> qtpu.HybridTensorNetwork:
    best_trial = min(study.best_trials, key=lambda trial: trial.values[0])
    htn = trial_to_hybrid_tn(best_trial)
    assert all(qtn.circuit.num_qubits <= SUBCIRCUIT_SIZE for qtn in htn.quantum_tensors)
    return htn

In [4]:
from benchmark.util import contraction_cost_log10, bruteforce_cost_log10
from qtpu.compiler.terminators import reach_num_qubits


def get_result(circuit: QuantumCircuit) -> dict[str, float]:
    study_only_wire = qtpu.compiler.hyper_optimize(
        circuit,
        terminate_fn=reach_num_qubits(SUBCIRCUIT_SIZE),
        compression_methods=["2q"],
        n_trials=100,
        max_cost=1e10,
        show_progress_bar=True,
    )
    study_normal = qtpu.compiler.hyper_optimize(
        circuit,
        terminate_fn=reach_num_qubits(SUBCIRCUIT_SIZE),
        n_trials=100,
        max_cost=1e10,
        show_progress_bar=True,
    )
    
    return {
        "qtpu": contraction_cost_log10(_min_cost_htn(study_normal)),
        "ckt": bruteforce_cost_log10(_min_cost_htn(study_normal)),
        "cutqc": bruteforce_cost_log10(_min_cost_htn(study_only_wire)),
        "scaleqc": contraction_cost_log10(_min_cost_htn(study_only_wire)),
    }

In [5]:
from benchmark.benchmarks import brick_ansatz, linear_ansatz, random_ansatz, cluster_ansatz

BENCHMARKS = [
    (f"cluster_{25}_8_1", cluster_ansatz([25]*8, 1)),
    (f"cluster_{25}_8_2", cluster_ansatz([25]*8, 2)),
    
    (f"linear_{NUM_QUBITS}_1", linear_ansatz(NUM_QUBITS, 1)),
    (f"linear_{NUM_QUBITS}_2", linear_ansatz(NUM_QUBITS, 2)),
    (f"linear_{NUM_QUBITS}_3", linear_ansatz(NUM_QUBITS, 3)),
    (f"brick_{NUM_QUBITS}_1", brick_ansatz(NUM_QUBITS, 1)),
    (f"brick_{NUM_QUBITS}_2", brick_ansatz(NUM_QUBITS, 2)),
    (f"brick_{NUM_QUBITS}_3", brick_ansatz(NUM_QUBITS, 3)),
]

In [6]:
from benchmark.util import append_to_csv

for name, circuit in BENCHMARKS * 5:
    result = get_result(circuit)
    append_to_csv(CSV_FILE, {"name": name, **result})

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

AttributeError: 'HybridTensorNetwork' object has no attribute 'qtns'

In [ ]:
df = pd.read_csv(CSV_FILE)
df = df.groupby("name").mean()
df.set_index(df.index.str.replace(f"_{NUM_QUBITS}_", "_"), inplace=True)
# reorder columns
df = df[["qtpu", "scaleqc", "ckt", "cutqc"]]


ax = df.plot.bar(rot=50)

# for ytick in ax.get_yticklabels():
    # ytick.set_text(f"$10^{{{ytick.get_text()}}}$")

ax.set_yticklabels([f"$10^{{{int(x.get_text())}}}$" for x in ax.get_yticklabels()])

NameError: name 'CSV_FILE' is not defined